## Multiple Chain 快速入门

Runnables 可以轻松地用来串联多个 Chains，使用 RunnablePassthrough 将输出同时传给多条后继链。

```
     Input
      / \
     /   \
 Chain1 Chain2
     \   /
      \ /
      Combine
```

本指南展示如何使用 Runnable 实现多个 AI 关于相同话题的辩论：

```
    输入话题
       |
       |
    原始观点
      / \
     /   \
 正面论述 反面论述
     \   /
      \ /
     最终总结
```

In [1]:
# 导入相关模块，包括运算符、输出解析器、聊天模板、ChatOpenAI 和 运行器
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# 创建一个计划器，生成一个关于给定输入的论证
planner = (
    ChatPromptTemplate.from_template("生成关于以下内容的论点: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

# 创建正面论证的处理链，列出关于基础回应的正面或有利的方面
arguments_for = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的正面或有利的方面"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

# 创建反面论证的处理链，列出关于基础回应的反面或不利的方面
arguments_against = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的反面或不利的方面"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

# 创建最终响应者，综合原始回应和正反论点生成最终的回应
final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "正面观点:\n{results_1}\n\n反面观点:\n{results_2}"),
            ("system", "给出批评后生成最终回应"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

# 构建完整的处理链，从生成论点到列出正反论点，再到生成最终回应
chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [2]:
chain.invoke({"input": "房地产低迷"})

'虽然房地产市场低迷可能会带来一些负面影响，如投资风险增加、利润下降等，但我们也不能忽视其带来的一些积极影响。在任何经济领域，都存在着波动和调整的必然性，房地产市场也不例外。房地产市场低迷期间，我们应该从多个角度出发，寻找解决问题的途径，如政府加强监管、企业加大创新、消费者理性购房等，共同努力促进房地产市场的健康发展。\n\n同时，房地产市场低迷也为购房者、开发商、投资者和政府提供了一些新的机遇和挑战。通过合理的政策引导、市场调节和行业自我调整，我们有信心克服困难，化挑战为机遇，推动房地产市场向更加健康、稳定的方向发展。在这个过程中，我们应该保持理性、审慎，共同努力促进房地产市场的长期稳定和可持续发展。'

#### 流式输出

In [3]:
## chain 最终输出经过了 StrOutputParser 处理，所以可以直接输出流式输出 s
for s in chain.stream({"input": "全球经济"}):
    print(s, end="", flush=True)

尽管全球经济发展面临一些挑战和问题，如经济霸凌、不公平贸易、资源过度消耗等，但我们不能否认全球经济的发展对于全球繁荣和稳定的积极影响。在应对这些问题的同时，我们应该以合作与共赢的态度来解决挑战，推动全球经济向着更加可持续和包容的方向发展。

通过加强国际合作、推动公平贸易、促进资源可持续利用、加强环境保护等措施，我们可以共同解决全球经济发展中的问题，实现经济社会的平衡发展。只有通过共同努力，我们才能实现全球经济的可持续发展，造福全人类。让我们携手合作，共同迈向更加繁荣、稳定和可持续的未来。

### Homework: 实现一个多链版本的代码生成，输入功能需求，输出2种（Python，Java）以上编程语言的代码实现。